# Stage 0: SETUP
The following libraries are used directly. For the full list of installed packages and versions, please see requirements.txt

In [1]:
# Imports
import pandas as pd
import numpy as np

# Stage 1: DATA ACQUISITION

Data is downloaded as csv files, and is already available in this repository in the data folder. See the readme for details on the source of the data.

## Case Data
Case data is provided with dates as columns. For processing, the data is melted into a "date" column and "case_count" column, and unused columns are dropped.

In [2]:
# Get raw data, only keep used columns
raw_case_data = pd.read_csv('data/raw/RAW_us_confirmed_cases.csv').drop(columns = ["Province_State", "Admin2", "UID", "iso2", "iso3", "Country_Region", "Lat", "Long_", "code3", "Combined_Key"])


# Pivot into dat column
case_data = raw_case_data.melt(id_vars = "FIPS", var_name="date", value_name = "Case_Count").dropna(subset=['FIPS'])

# Cast types
case_data["date"] = pd.to_datetime(case_data["date"])
case_data["FIPS"] = case_data["FIPS"].astype(int).astype(str).str.pad(5, 'left', '0')
case_data.head()


,FIPS,date,Case_Count
0,01001,2020-01-22,0
1,01003,2020-01-22,0
2,01005,2020-01-22,0
3,01007,2020-01-22,0
4,01009,2020-01-22,0


## Compliance Data


In [3]:
# Get raw data
raw_compliance_data = pd.read_csv('data/raw/mask_use_by_county.csv')
raw_compliance_data["FIPS"] = raw_compliance_data['COUNTYFP'].astype(str).str.pad(5, 'left', '0')

# Combine into single score
raw_compliance_data["mask_compliance"] = np.dot(raw_compliance_data.iloc[:,1:6], [0, 0.25, 0.5, 0.75, 1])

# Clean
compliance_data = raw_compliance_data[["FIPS", "mask_compliance"]]
compliance_data.head()

,FIPS,mask_compliance
0,01001,0.75075
1,01003,0.74200
2,01005,0.73200
3,01007,0.83700
4,01009,0.72300


## Housing Data

In [4]:
# Get raw data, only keep used columns
use_cols = ["fips2010", "n_units", "li_units"]
raw_housing_data = pd.read_csv('data/raw/LIHTCPUB.CSV', usecols=use_cols)
raw_housing_data["FIPS"] = raw_housing_data["fips2010"].str[:5]

housing_data = raw_housing_data.drop(columns=["fips2010"]).groupby("FIPS").sum()
housing_data.head()

,n_units,li_units
FIPS,,
01001,499.0,499.0
01003,1963.0,1920.0
01005,413.0,413.0
01007,278.0,278.0
01009,275.0,275.0


## Demographic Data

In [5]:
# INCOME
# Get raw data, only keep used columns
use_cols = ["id", "Estimate!!Households!!Total", "Estimate!!Households!!Mean income (dollars)"]
raw_income_data = pd.read_csv('data/raw/income_census_data_2019.csv', header=1, usecols=use_cols)
raw_income_data.columns= ["FIPS", "household_count", "mean_income"]
raw_income_data["FIPS"] = raw_income_data["FIPS"].str[-5:]

# POPULATION
# Get raw data, only keep used columns
use_cols = ["id", "Estimate!!SEX AND AGE!!Total population"]
raw_population_data = pd.read_csv('data/raw/population_census_data_2019.csv', header=1, usecols=use_cols)
raw_population_data.columns= ["FIPS", "population"]
raw_population_data["FIPS"] = raw_population_data["FIPS"].str[-5:]

# Combine
demographic_data = raw_income_data.merge(raw_population_data, on="FIPS", how='inner')
demographic_data.head()

,FIPS,household_count,mean_income,population
0,13013,27765,77081,83240
1,13015,39742,70644,107738
2,13021,56726,67678,153159
3,13031,28660,61191,79608
4,13045,42798,69895,119992


# Stage 2: DATA PROCESSING

Merge the data on FIPS

In [6]:
# Combine on FIPS
combined_data = case_data.merge(compliance_data, on="FIPS", how='left')
combined_data = combined_data.merge(housing_data,on="FIPS", how='left')
combined_data = combined_data.merge(demographic_data,on="FIPS", how='left')
combined_data.head()

,FIPS,date,Case_Count,mask_compliance,n_units,li_units,household_count,mean_income,population
0,01001,2020-01-22,0,0.75075,499.0,499.0,NaN,NaN,NaN
1,01003,2020-01-22,0,0.74200,1963.0,1920.0,NaN,NaN,NaN
2,01005,2020-01-22,0,0.73200,413.0,413.0,NaN,NaN,NaN
3,01007,2020-01-22,0,0.83700,278.0,278.0,NaN,NaN,NaN
4,01009,2020-01-22,0,0.72300,275.0,275.0,NaN,NaN,NaN


Add extra calculations per fip

In [32]:
# Assumed variables
active_window = 14 # Days after infection that case is active

# Calculations per FIP
combined_data.sort_values(by="date", inplace=True)

# Do rolling average of cases to account for weekly fluctiation
grouped = combined_data.groupby("FIPS")
combined_data["Case_Count_7da"] = grouped["Case_Count"].rolling(window=7).mean().reset_index(0,drop=True)
grouped = combined_data.groupby("FIPS")
combined_data["new_cases"] = grouped["Case_Count_7da"].diff().reset_index(0,drop=True).rolling(window=7).mean()
grouped = combined_data.groupby("FIPS")
combined_data["active_cases"] = grouped["new_cases"].rolling(window=active_window).sum().reset_index(0,drop=True)
combined_data["vulnerable_pop"] = combined_data["population"] - combined_data["Case_Count_7da"]

combined_data[combined_data["FIPS"] == '13135'].head()

,FIPS,date,Case_Count,mask_compliance,n_units,li_units,household_count,mean_income,population,Case_Count_7da,new_cases,active_cases,vulnerable_pop
477,13135,2020-01-22,0,0.8355,4746.0,2810.0,301471.0,93401.0,936250.0,NaN,NaN,NaN,NaN
3809,13135,2020-01-23,0,0.8355,4746.0,2810.0,301471.0,93401.0,936250.0,NaN,NaN,NaN,NaN
7141,13135,2020-01-24,0,0.8355,4746.0,2810.0,301471.0,93401.0,936250.0,NaN,NaN,NaN,NaN
10473,13135,2020-01-25,0,0.8355,4746.0,2810.0,301471.0,93401.0,936250.0,NaN,NaN,NaN,NaN
13805,13135,2020-01-26,0,0.8355,4746.0,2810.0,301471.0,93401.0,936250.0,NaN,NaN,NaN,NaN


Filter down to jsut the Atlanta area counties

In [117]:
# Filter down to Atlanta counties
county_fips = [
    '13057', #Cherokee
    '13063', #Clayton
    '13067', #Cobb
    '13089', #DeKalb
    '13097', #Douglas
    '13113', #Fayette
    '13117', #Forsyth
    '13121', #Fulton
    '13135', #Gwinettt
    '13151', #Henry
    '13247', #Rockdale
]
combined_data_atl = combined_data[combined_data["FIPS"].isin(county_fips)].copy()
combined_data_atl.dropna(subset=["active_cases", "population", "vulnerable_pop", "new_cases", "FIPS"])
combined_data_atl.head()

,FIPS,date,Case_Count,mask_compliance,n_units,li_units,household_count,mean_income,population,Case_Count_7da,new_cases,active_cases,vulnerable_pop
441,13063,2020-01-22,0,0.81925,3933.0,1849.0,97030.0,60225.0,292256.0,NaN,NaN,NaN,NaN
438,13057,2020-01-22,0,0.80725,2907.0,775.0,93441.0,101950.0,258773.0,NaN,NaN,NaN,NaN
453,13089,2020-01-22,0,0.86550,14612.0,5618.0,289829.0,94336.0,759297.0,NaN,NaN,NaN,NaN
466,13113,2020-01-22,0,0.75550,569.0,337.0,41253.0,128190.0,114421.0,NaN,NaN,NaN,NaN
443,13067,2020-01-22,0,0.81100,6727.0,2278.0,286952.0,108459.0,760141.0,NaN,NaN,NaN,NaN


# Stage 3: MODELING

Here we model infection rates and case counts. The model is 

$$
C_{new} = P_v * (1 - (1 - r)^{\frac{C_{active}}{P_{total}}x})
$$

Where $C_{new}$ and $C_{active}$ are new and active cases, $P_v$ and $P_{total}$ are vulnerable and total population, r is the transmission rate and x is the exposure rate.

In [76]:
def transmissionModel(data, r, x):
    c_active = data["active_cases"]
    p_total = data["population"]
    p_vuln = data["vulnerable_pop"]
    daily_exposures = c_active / p_total * x
    return p_vuln * (1 - pow( 1 - r,daily_exposures))

We fit r and x to a dataset. r is bound between 0 and 1, since it's a probability, and x is bounded from 1 to 1000 to help in optimizing the curve fit.

In [113]:
from scipy.optimize import curve_fit

def getTransmissionVariables(data):
    y = data["new_cases"]
    X = data[["active_cases", "population", "vulnerable_pop"]]
    popt, pcov = curve_fit(transmissionModel, X, y, p0 = [0.1, 10], bounds=(0, [1,1000]))
    return popt


For each month in each county, we calculate the values

In [124]:
combined_data_atl["month"] = combined_data_atl["date"].dt.strftime('%y-%m')
model_fits = model_data.groupby(["FIPS", "month"]).apply(getTransmissionVariables)
model_fits.columns = ["x"]
model_fits = pd.DataFrame(model_fits.tolist(), index=model_fits.index, columns = ["transmission_chance", "exposure_rate"])
modeled_data_atl = combined_data_atl.merge(model_fits, on=["FIPS", "month"])
modeled_data_atl.head()

,FIPS,date,Case_Count,mask_compliance,n_units,li_units,household_count,mean_income,population,Case_Count_7da,new_cases,active_cases,vulnerable_pop,month,transmission_chance,exposure_rate
0,13063,2020-02-01,0,0.81925,3933.0,1849.0,97030.0,60225.0,292256.0,0.0,0.0,NaN,292256.0,20-02,0.1,10.0
1,13063,2020-02-02,0,0.81925,3933.0,1849.0,97030.0,60225.0,292256.0,0.0,0.0,NaN,292256.0,20-02,0.1,10.0
2,13063,2020-02-03,0,0.81925,3933.0,1849.0,97030.0,60225.0,292256.0,0.0,0.0,NaN,292256.0,20-02,0.1,10.0
3,13063,2020-02-04,0,0.81925,3933.0,1849.0,97030.0,60225.0,292256.0,0.0,0.0,NaN,292256.0,20-02,0.1,10.0
4,13063,2020-02-05,0,0.81925,3933.0,1849.0,97030.0,60225.0,292256.0,0.0,0.0,NaN,292256.0,20-02,0.1,10.0


In [ ]:
calculateSpreadRate(neighbor_data)["mask_mandate"].plot()

In [ ]:
import matplotlib.pyplot as plt
fig,ax = plt.subplots()
fig.set_size_inches(10, 6, forward=True)
#fig.autofmt_xdate()

# PLot rates
county_rates["infection_rate_change_smooth"].plot(label="Gwinnet County", color="orange")
neighbor_rates["infection_rate_change_smooth"].plot(label="Neighbors", color="slateblue", alpha=0.6, linestyle='dotted')
plt.ylabel("Cases per Mil. Vuln. Pop")
plt.xlabel("")
plt.legend()

#PLot mask usage
ax2=ax.twinx()
neighbor_rates["mask_mandate"].plot.area(color="skyblue", alpha=0.2, label="% Neighbors with Mask Mandate")
plt.axis('off')
plt.legend()

# Annotation
plt.title(label = "Change in Infection Rate")
#plt.show()
plt.savefig('Change_in_infection_rate.png')


In [ ]:
fig,ax = plt.subplots()
fig.set_size_inches(10, 6, forward=True)

# Find diff
infection_rate_diff = (county_rates["infection_rate_7da"] - neighbor_rates["infection_rate_7da"]) * 1000000
zero = infection_rate_diff * 0

# PLot rates
infection_rate_diff.plot(label="Infection Rate Difference", color="orange")
zero.plot(color="black", linewidth=0.5, linestyle='dashed', alpha=0.5)
plt.ylabel("Cases per Mil. Vuln. Pop")
plt.xlabel("")

#PLot mask usage
ax2=ax.twinx()
neighbor_rates["mask_mandate"].plot.area(color="skyblue", alpha=0.2, label="% Neighbors with Mask Mandate")
plt.axis('off')
plt.legend()

#Anotate
plt.title(label = "Diff in Daily New Cases Between Gwinnett County and Neighbors")
#plt.show()
plt.savefig('Diff_in_infection_rate.png')